01 - Generate JSON from Post-Inclusion/Exclusion CSV
- Enter path to CSV which has successful articles in the index column
- Enter path to JSON which has the articles with labeled sections

In [ ]:
csv_path = "/Users/cu135/Library/CloudStorage/OneDrive-Personal/OneDrive_Documents/Work/Software/Research/nimlab/gpt_document_reader/amnesia_cases/ocr/keyword_matching_labeled_text/text_evaluations/inclusion_exclusion_results/unfiltered_results.csv"
json_path = "/Users/cu135/Library/CloudStorage/OneDrive-Personal/OneDrive_Documents/Work/Software/Research/nimlab/gpt_document_reader/amnesia_cases/ocr/keyword_matching_labeled_text/labeled_sections.json"

In [ ]:
from calvin_utils.gpt_sys_review.json_utils import FilterPapers

# Initialize and run the FilterPapers class
filter_papers = FilterPapers(csv_path=csv_path, json_path=json_path)
filter_papers.run()

02 - Prepare Data Extraction Questions
- Dev Note: This is a repeat class and should be inherited from a .py

In [ ]:
from calvin_utils.gpt_sys_review.gpt_utils import QuestionTemplate

question_template = QuestionTemplate()
question_template.data_extraction_questions()

Copy the Template Dict from Above and Fill it Out As per the Example

In [ ]:
question = {
    "How affected is this study by selection bias? Little (Good). Very (Bad). Unsure (Unclear).": "selection_bias",
    "How well was the timeline of lesion/symptom onset described? (Good/Bad/Unclear)": "exposure",
    "How affected is this case by attribution error? Little (Good). Very (Bad). Unsure (Unclear).": "outcome",
    "How reasonable is the attribution of the lesion/diagnosis to the symptom? (Good/Bad/Unclear)": "attribution_error",
    "How well was the patients baseline pre-lesion described? (Good/Bad/Unclear)": "pre_event_assessment",
    "How well was the patients outcome post-lesion described? (Good/Bad/Unclear)": "post_event_assessment",
    "Do you think the neuroimaging was taken within temporal proximity to the lesion? Days-weeks (Good). Months-years (Bad). Unsure (Unclear).": "lesion_related-images",
    "Do you think another practitioner would come to the same conclusion (diagnosis/attribution) that this group did? (Good/Bad/Unclear)": "replicability",
    "How is the quality of this case overall? (Good/Bad/Unclear)": "overall_appraisal"
}

03 - Extract Data with GPT


In [ ]:
# Define file paths
api_key_path = "/Users/cu135/Library/CloudStorage/OneDrive-Personal/OneDrive_Documents/Work/Software/Research/nimlab/openai_key.txt"

# Define the keys you want to consider (exclude 'References')
keys_to_consider = ["Introduction", "Methods", "Results", "Discussion", "Conclusion"]  # Add or remove keys as per your requirement

# Define the type of article and questions
article_type = "research"

In [ ]:
from calvin_utils.gpt_sys_review.gpt_utils import OpenAIChatEvaluator

In [ ]:
evaluator = OpenAIChatEvaluator(api_key_path, json_path, keys_to_consider, article_type, question)
answers = evaluator.evaluate_all_files()
evaluator.save_to_json(answers)

04 - Convert results to a CSV

In [1]:
import numpy as np

#Map responses to data
# Define a dictionary to map keywords in order or priority to values
keyword_mapping = {
    np.nan: ["unclear", "unsure", "un"],
    0: ["poor", "bad", "negative"],
    1: ["good", "excellent", "positive"]
}


In [2]:
json_path = r'/Users/cu135/Library/CloudStorage/OneDrive-Personal/OneDrive_Documents/Work/Software/Research/nimlab/gpt_document_reader/amnesia_cases/ocr/keyword_matching_labeled_text/text_evaluations/data_extraction_results_strict/research_evaluations.json'

In [3]:
from calvin_utils.gpt_sys_review.json_utils import CustomSummarizer

# Create an instance of CustomSummarizer
custom_summarizer = CustomSummarizer(json_path=json_path, keyword_mapping=keyword_mapping)

# Run the custom summarization and save the results to CSV
result_df = custom_summarizer.run_custom()

# Display the resulting DataFrame
print(result_df)

/Users/cu135/.local/lib/python3.10/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


                                                    Does the patient(s) represent(s) the whole experience of the investigator (center) or is the selection method unclear to the extent that other patients with similar presentation may not have been reported? (Good/Bad/Unclear)  \
Mosimann et al. - 2012 - Fornix infarction and ...                                                NaN                                                                                                                                                                                  
Parkin - IMPAIRMENT OF MEMORY FOLLOWING DISCRET...                                                NaN                                                                                                                                                                                  
Amnesia after right frontal subcortical lesion,...                                                NaN                                                           